# Manual testing of modules

## Identification Module

In [2]:
import os
import pickle
from openai import OpenAI
from google import genai
from google.genai import types as genTypes

from vif.feature_identification.feature_identification import SimpleGeminiIdentificationModule
RES_PATH = "/home/creux/Documents/AI/VIFagent/tests/resources/seg"
DEBUG_PATH = "/home/creux/Documents/AI/VIFagent/notebooks/oracle_verification/.debug"
get_res = lambda file: os.path.join(RES_PATH,file)

""" client=OpenAI(
        api_key=os.environ.get("OPENROUTER_API_KEY"),
        base_url="https://openrouter.ai/api/v1",
    ) """
client=genai.Client(
        api_key=os.environ.get("GOOGLE_API_KEY"),
        http_options=genTypes.HttpOptions(api_version='v1alpha')
    )

simple_identification_module = SimpleGeminiIdentificationModule(
    client=client,
    model="gemini-2.5-pro",
    temperature=0.3,
)
from PIL import Image
original_image = Image.open(get_res("rgb_stc.png"))

In [5]:
simple_identification_module.debug_instance_creation(True, DEBUG_PATH)

simple_identification_module.get_features(original_image)

['Top light purple square',
 'Bottom left light red triangle',
 'Bottom right light green circle']

In [6]:

simple_identification_module.debug_instance_creation(True, DEBUG_PATH)

# first feature search phase works skipping
features = [
    "White background",
    "Top light purple square",
    "Bottom-left light red triangle",
    "Bottom-right light green circle",
]
original_image = original_image.convert("RGB")
feat_seg = simple_identification_module.segments_from_features(features, original_image)

In [7]:
with open("/home/creux/Documents/AI/VIFagent/notebooks/oracle_verification/.debug/identification/tmpfiles/segments.pickle","wb") as sf:
    pickle.dump(feat_seg,sf)

## Oracle creation

In [8]:

from vif.falcon.oracle.guided_oracle.guided_code_oracle import OracleGuidedCodeModule
with open("/home/creux/Documents/AI/VIFagent/notebooks/oracle_verification/.debug/identification/tmpfiles/segments.pickle","rb") as sf:
    feat_seg = pickle.load(sf)
oracle_module = OracleGuidedCodeModule(
    model="gemini-2.5-pro",
    temperature=0.3,
    client=OpenAI(
        api_key=os.environ.get("GOOGLE_API_KEY"),
        base_url="https://generativelanguage.googleapis.com/v1beta/openai/",
    ),
)
instruction = "Rotate the square by 45 degree clockwise"
oracle_module.debug_instance_creation(True, DEBUG_PATH)
oracle = oracle_module.get_oracle(feat_seg,instruction,original_image,simple_identification_module.segments_from_features)

/home/creux/Documents/AI/VIFagent/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
You are trying to use a model that was created with Sentence Transformers version 5.0.0, but you're currently using version 4.1.0. This might cause unexpected behavior or errors. In that case, try to update to the latest version.


In [ ]:
from vif.falcon.oracle.oracle import OracleResponse


modified_image = Image.open("/home/creux/Documents/AI/VIFagent/tests/resources/seg/stc_rgb_rotc.png")
or_res:OracleResponse = oracle(modified_image)

In [ ]:
or_res.evaluation_code

In [ ]:
or_res.feedbacks

## Edition Module